In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from IPython.core.display import display, HTML
import os

from src import *
pd.set_option('display.max_rows', None)

# Parameters

These parameters configure various things such as the relative location of data files, which date is being analysed
and the assumed generation period for an infection.

In [ ]:
ALL_VIC_CASES='archive/2021-07-12/all-vic-cases.csv'
QUARANTINE='archive/2021-07-16/quarantine.csv'
GENERATION_DAYS=5
ENABLE_MELBOURNE_ANIMATION=False

TODAY=datetime.today().strftime("%Y-%m-%d")
YESTERDAY=add_days(TODAY, -1)
LAST_WEEK=add_days(TODAY, -7)
RUN_DATE=TODAY
LOAD_DATE=RUN_DATE

PREV_DAY=add_days(RUN_DATE, -1)
NEXT_WEEK=add_days(RUN_DATE, 7)
PREV_WEEK=add_days(RUN_DATE, -7)
PREV_FORTNIGHT=add_days(RUN_DATE, -14)

NSW_6_MONTHS=lambda date: f"archive/{date}/last-6-months-nsw.csv"
NSW_14_DAYS=lambda date: f"archive/{date}/last-14-days-nsw.csv"

AMEND_TOTAL=None

# Data Preparation

**sweep_downloads()** moves files from the $HOME/Downloads directory into today's archive directory. Thses should first be
downloaded from the "Last 14 days (new)" and "Last 6 months (true)" panels of the NSW Transmission Sources section of [covid19data.com.au](https://www.covid19data.com.au/nsw)

In [ ]:
sweep_downloads(TODAY)
if not os.path.exists(NSW_6_MONTHS(LOAD_DATE)):
    LOAD_DATE=YESTERDAY
assert os.path.exists(NSW_6_MONTHS(LOAD_DATE))

In [ ]:
nsw_df = update_df(load_data(NSW_6_MONTHS(LOAD_DATE)),load_data(NSW_14_DAYS(LOAD_DATE)))
if AMEND_TOTAL:
    nsw_df = append_recent_date(nsw_df, RUN_DATE, AMEND_TOTAL)
vic_df = load_vic_data(ALL_VIC_CASES)
quarantine_df = load_quarantine(QUARANTINE)

Next, we truncate and refindex the data frames to each outbreak.

In [ ]:
avalon = select_outbreak(nsw_df[(nsw_df['date'] >= '2020-12-17') & (nsw_df['date'] <= '2021-01-16')], generation_days=GENERATION_DAYS)
bondi = select_outbreak(nsw_df[(nsw_df['date'] >= '2021-06-17')], generation_days=GENERATION_DAYS)
vic_outbreak = select_outbreak(vic_df[(vic_df['date'] >= '2020-05-27') & (vic_df['date'] <= '2020-10-29')], generation_days=GENERATION_DAYS)

In [ ]:
display(HTML(summary(bondi)))

# Growth Model

If the model was perfect, tbe following would be true:

$
\space\space\space\space\space\text{cumulative}_{t+1} = (1+\frac{\text{g}_t}{100})\times\text{cumulative}_t
$

In practice, the growth rate, $g_t$, is estimated by fitting a linear regession through a set of points:

$
\space\space\space\space\space(t-k, \ln{(\text{cumulative}_{t-k})})
$

for k = 0..4. This yields parameters of a linear reqregssion:

$
\space\space\space\space\space\ln(\text{cumulative}_t) = m \times t + b
$

Raisng e to each side, yields:

$
\space\space\space\space\space\text{cumulative}_t = (e^m)^t \times e^b = e^m \times (e^m)^{t-1} \times e^b = e^m \times \text{cumulative}_{t-1}
$

The growth rate, $g_t$, is thus:

$
\space\space\space\space\space g_t = (e^m-1) * 100
$

The minimum growth rate $g_{t,min}$ is defined as:

$
\space\space\space\space\space g_{t,min} = min(\{k: 0 \le k < 5: g_{t-k}\})
$

The 7-day forward projection of the cumulative total, $projection_t$ used on the so-called "_hedgehog plot_" uses $g_{t,min}$ since this was observed to provide closest fit to the Melbourne 2020 outbreak, particularly in the later stages. 

$
\space\space\space\space\space \text{projection}_t = (1+\frac{g_{t-7,min}}{100})^7 \times \text{cumulative}_{t-7}
$

The intuitive justification is that because the growth rate eventually starts to decay, a fit to the last 5 days growth is going to tend overestimate the growth in the next 5 days so choosing the minimum observed growth rate estimate is more likely to be closer to true growth rate rather than the very latest estimate.

The replication factor, $R_{eff}$ is defined as:

$
\space\space\space\space\space R_{eff} = (1+\frac{g_t}{100})^5
$

The doubling period, in days, is defined as:

$
\space\space\space\space\space \text{doubling period}_t = \ln{(2)}/\ln{(1+\frac{g_t}{100})}
$

The table below documents various calculated parameters.

- **date** - the reporting date
- **cumulative** - the cumulative cases to the reporting date. This is usually the day after the cases were "notified".
- **total** - the number of cases reported on the reporting date
- **ols-growth-rate** - $g_t$, growth rate obtained from exponential fit to previous 5-days cumulative amounts expressed as a percentage change of cumulative cases per day
- **ols-growth-rate-min** - $g_{t,min}$, minimum ols-growth-rate statistic calculated in previous 5 days
- **ols-growth-rate-decay** - an expoenential fit to the ols-growth-rate statistic in the previous 5 days expressed as a percentage change of the ols-growth-date per day
- **doubling-period** - doubling period (in days) implied by ols-growth-rate
- **Reff** - implied $R{eff}$, assuming a 5 day generation period

In [ ]:
display(bondi[[
    "date",
    "cumulative", 
    "total", 
    "ols-growth-rate", 
    "ols-growth-rate-min", 
    "ols-growth-rate-decay",  
    "linear-growth-rate", 
    "doubling-period",
    "Reff",
    "ltlc-gradient"
]].tail(21))

# Linear Growth 

## Sydney 2021

In [ ]:
ax=plot_linear_growth(bondi)

In [ ]:
ax=plot_linear_growth_error(bondi)

## Melbourne 2020

In [ ]:
ax=plot_linear_growth(vic_outbreak)

In [ ]:
ax=plot_linear_growth_error(vic_outbreak)

# 7 Day Model

This section presents the 7-day old, 7-day forward projection, e.g. the prediction for the current date and a comparison with what actually happened. This is about measuring the effectiveness for a 7-day model for a known
period. In particular, we are not trying to predict what will happen in the next 7 days/

- **date** - the reporting date
- **cumulative** - the cumulative cases to the reporting date. This is usually the day after the cases were "notified".
- **total** - the number of cases reported on the reporting date
- **7-day-delta** - 7-day change in cumulative cases
- **7-day-projection** - the 7-day old projection of the cumulative total for the reporting date
- **7-day-projection-error** - the error in 7-day old projection. +ve is projection overshoot.
- **7-day-projection-relative-error** - the relative error in the projected cumulative change vs the actual cumulative change expressed as a percentage

In [ ]:
display(bondi[[
    "date",
    "cumulative", 
    "total", 
    "7-day-delta",
    "7-day-projection",
    "7-day-projection-error",
    "7-day-projection-relative-error",
]].tail(21))

# 1-day Projections (Past and Present)

- **date** - the reporting date
- **cumulative** - the cumulative cases to the reporting date. This is usually the day after the cases were "notified".
- **total** - the number of cases reported on the reporting date
- **one-day-projection-cumulative** - the 1-day cumulative projection
- **one-day-projection-total** - the 1-day total projection
- **one-day-error** - the difference between the 1-day projection and the actual
- **one-day-relative** - the ratio of error of the projection from the daily total expressed as a percentage.


In [ ]:
display(bondi[[
    "date",
    "cumulative", 
    "total", 
    "one-day-projection-cumulative", 
    "one-day-projection-total",
    "one-day-error", 
    "one-day-relative-error", 
]].tail(21))

# Cumulative Plot (Partial)



In [ ]:
output=pd.DataFrame(columns=['cumulative', 'min', 'vic'])
output[['min', 'cumulative']] = bondi[['min','cumulative']]
output=output.reindex([r for r in range(0, len(bondi)+14)])
output['vic'] = vic_outbreak['cumulative']
x=11
output['vic-offset'] = vic_outbreak['cumulative'].shift(-x)
ax=output.plot(figsize=(10,10))
ax.set_yscale('log')
ax.grid()
ax.set_title(f'Cumulative Case Growth Projections ({RUN_DATE})')
ax.legend([
    'Sydney (2021) - actual', 
    'Sydney (2021) - model', 
    'Melbourne (2020)',
    f'Melbourne (2020) +{x} days'
])
ax.figure.savefig(f'archive/{RUN_DATE}/cumulative-partial.png')
_=_

In [ ]:
df=vic_outbreak
ax=df[['min', 'cumulative']].plot(figsize=(10,10))
#ax.set_yscale('log')
ax.grid()
ax.plot(bondi['cumulative'])
ax.plot(bondi['min'])
ax.legend(['model (Melbourne 2020) ', 'cumulative (Melbourne 2020)','cumulative (Sydney 2021)',  'model (Sydney 2021) '])
ax.set_title("7 Day Projection vs Actual (Melbourne 2020, Sydney 2021)")
ax.figure.savefig(f'archive/{RUN_DATE}/cumulative-full.png')
_=_

In [ ]:
VIC_EXTRA_DAYS=0

vic_growth_params=derive_growth_params(vic_outbreak[(vic_outbreak.index >= 70) & (vic_outbreak.index < 120)], generation_days=GENERATION_DAYS)
bondi_growth_params=derive_growth_params(bondi.tail(8), generation_days=GENERATION_DAYS)
N=1
bondi_growth_params_3=derive_growth_params(bondi.tail(8+N).head(8), generation_days=GENERATION_DAYS)
gp=derive_growth_params(bondi[(bondi.index>15)])
bondi_projection_1=select_outbreak(project_ols_growth_rate_min(bondi, 84, vic_growth_params[1]))
bondi_projection_2=select_outbreak(project_ols_growth_rate_min(bondi, 110, gp[1]))
bondi_projection_3=select_outbreak(project_ols_growth_rate_min(bondi.head(len(bondi)-N), 84+N, gp[1]))


vic_partial=vic_outbreak.head(len(bondi)+VIC_EXTRA_DAYS)
vic_partial_growth_params=derive_growth_params(vic_partial)
vic_projection=select_outbreak(project_ols_growth_rate_min(vic_partial, len(vic_outbreak)-len(bondi)-VIC_EXTRA_DAYS, vic_partial_growth_params[1]), generation_days=GENERATION_DAYS)

# Growth Plot

This plot shows the estimated and projected daily cumulative growth rate as a percentage. By definition, the cumulative growth rate is always an (eventually small) non-negative value. The values plotted for each outbreak are $g_t$ and $g'_t$ as calculated above.

In [ ]:
gp=GrowthPlot(RUN_DATE)
gp.add(bondi, offset=0, legend="Sydney 2021")
#gp.add(avalon, offset=0, legend="Avalon 2020")
gp.add(vic_outbreak, offset=0, legend="Melbourne 2020")
gp.ax.plot(bondi['ols-growth-rate-min'],color="C3")
gp.ax.plot(vic_outbreak['ols-growth-rate-min'], color="C4")
gp.ax.plot(bondi_projection_1['ols-growth-rate-min'], linestyle='dotted', color='C4')
#gp.ax.plot(bondi_projection_2['ols-growth-rate-min'], linestyle='dashed', color='C3')
#gp.ax.plot(bondi_projection_3['ols-growth-rate-min'], linestyle='dotted', color='C5')
gp.ax.set_yscale('log')
gp.legend = gp.legend+[
    'Sydney 2021 (retrospective model)', 
    'Melbourne 2020 (retrospective model)', 
    'Sydney 2021 (projection - Melbourne 2020 decay)',
#    f'Sydney 2021 (projection - Sydney 2021 decay ({RUN_DATE})',
#    f'Sydney 2021 (projection - Sydney 2021 decay ({PREV_DAY})'
]
gp.ax.legend(gp.legend)
gp.ax.figure.savefig(f'archive/{RUN_DATE}/cumulative-growth.png')

#gp.add(vic_outbreak.shift(-11), offset=0, legend="Melbourne 2020 (shifted)")
_=_

# Growth Projection Relative Error Plot

This plot display the relative error between the projected 7-day cumulative growth and the growth that occurred in practice.

The relative error is defined as:

$
\space\space\space\text{error}_t = \text{projection}_t - \text{cumulative}_t
$

$
\space\space\space\text{relative error}_t = \frac{\text{projection}_t - \text{cumulative}_t}{\text{cumulative}_t - \text{cumulative}_{t-7}} \times 100
$


In [ ]:
output=pd.DataFrame()
output['vic'] = modeling_errors(vic_outbreak)
output["vic"]
output["bondi"] = modeling_errors(bondi)
#output["avalon"] = modeling_errors(avalon)
                                    
ax=output.loc[output.index >= 15, ['bondi', 'vic', ]].plot(figsize=(10,10))
ax.grid()                         
ax.set_title("Growth Projection Relative Error % vs Day Of Outbreak")
ax.set_xlabel("Day Of Outbreak")
ax.set_ylabel("% overshoot of projection vs actual")
ax.legend(['Sydney 2011', 'Melbourne 2020'])
ax.figure.savefig(f'archive/{RUN_DATE}/modellng-error.png')
_=_

In [ ]:
plot_vic=True
GRAPH_DATE=bondi.tail(1).date.values[0]
pp = PhasePlot(f"New Cases vs Error Modeling % - Sydney 2021 vs Melbourne 2020 - ({GRAPH_DATE})")

bondi_idx=pp.add(bondi, offset=20, legend="Sydney 2021", color="C0") # 20
if plot_vic:
    vic_idx=pp.add(vic_outbreak, offset=15, legend="Melbourne 2020", color="C1") # 15

pp.add_horizon(horizon(bondi, 7), legend=f"7-Day Projection for {NEXT_WEEK}", color="blue")
pp.add_horizon(horizon(bondi.head(len(bondi)-7), 7), legend=f"7-Day Projection for {RUN_DATE}", color="red")
# for i in range(0,7):
#     pp.add_horizon(horizon(bondi.head(len(bondi)-7-i),7), legend=f"today - as projected {7+i} days ago", color=f"C{i}")


if plot_vic:
    first_case=pp.frames[vic_idx].head(1)['date']
    last_case=pp.frames[vic_idx].tail(1)[['date', 'total', '7-day-projection-relative-error']]

    pp.add_label(vic_idx, "2020-08-04", "peak daily new cases")
    pp.add_label(vic_idx, "2020-07-16", "last model undershoot, prior to recovery")
    pp.add_label(vic_idx, "2020-08-02", "stage 4 restrictions announced")
    pp.add_label(vic_idx, "2020-07-15", "similar state (VIC)")
    pp.add_label(vic_idx, "2020-07-09", "stage 3 restrictions announced")
    pp.add_label(vic_idx, last_case.values[0][0], f"last day plotted (VIC)")
    pp.add_label(vic_idx, first_case.values[0], f"first day plotted (VIC)")

    # pp.add_label(vic_idx, "2020-08-09", "1 week after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-08-16", "2 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-08-23", "3 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-08-30", "4 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-09-06", "5 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-09-13", "6 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-09-20", "7 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-09-27", "8 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-10-03", "9 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-10-10", "10 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-10-17", "11 weeks after stage 4 restrictions announced")
    # pp.add_label(vic_idx, "2020-10-24", "12 weeks after stage 4 restrictions announced")

first_case_nsw=pp.frames[bondi_idx].head(1)['date']
pp.add_label(bondi_idx, GRAPH_DATE, "current state (NSW)")
pp.add_label(bondi_idx, PREV_WEEK, "a week ago (NSW)")
pp.add_label(bondi_idx, PREV_FORTNIGHT, "two weeks ago (NSW)")
pp.add_label(bondi_idx, first_case_nsw.values[0], f"first day plotted (NSW)")    
    
pp.ax.figure.savefig(f'archive/{RUN_DATE}/hedgehog.png')
_=_

# Derivation Of Decay Rate

The decay rate is calculated as an exponential fit through growth rates after July 2 (day 15).

In [ ]:
gp=derive_growth_params(bondi[(bondi.date >= '2021-07-02')])
ax=bondi["ols-growth-rate"].plot(figsize=(10,10))
ax.plot((gp[1]**bondi.index)*gp[0])
ax.set_title(f"Daily Cumulative Growth Rate % (Sydney 2021) ({RUN_DATE})")
ax.set_xlabel("Day Of Outbreak (t)")
ax.set_ylabel("Daily Cumulative Growth Rate % (gt)")
ax.legend(["observed", f"trend: gt=g0 * (1+r/100)^t"])
ax.text(34, 62, f"g0={round(gp[0],3)}, r={round((gp[1]-1)*100,3)}")
ax.grid()
ax.figure.savefig(f'archive/{RUN_DATE}/growth-rate-trend.png')

# Sydney Derivatives

This plots the actual cumulative <span style="color:blue">blue</span>) and daily totals (<span style="color:orange">orange</span>) for the Sydney 2021 outbreak together
with the implied daily growth rate (<span style="color:green">green</span>) and the rate of decay of the growth rate (<span style="color:red">red</red>).

The dotted extensions of each observed plot are projections into the future assuming a constant (negative) decay rate. This decay rate is based on expoential fit through recent growth rate estimates (see plot above). This assumption is not sound and is likely to underestimate the true decay rate particularly in the later stages of the outbreak. As such peak estimates and timing are indicative only and are likely to be worst-case by a factor of 2 or more.

In [ ]:
df=select_outbreak(project_ols_growth_rate_min(bondi, 200-len(bondi), gp[1]))
ax=plot_derivatives(df, len(bondi), "Sydney 2021")
ax.figure.savefig(f'archive/{RUN_DATE}/derivatives-sydney-partial.png')

In [ ]:
df=vic_partial
gp=derive_growth_params(df[df.index>30])
ax=df["ols-growth-rate"].plot(figsize=(10,10))
ax.plot((gp[1]**bondi.index)*gp[0])
ax.set_title(f"Daily Cumulative Growth Rate % (Melbourne 2021) ({RUN_DATE})")
ax.set_xlabel("Day Of Outbreak")
ax.set_ylabel("Daily Cumulative Growth Rate %")
ax.legend(["observed", f"trend: y={round(gp[0],3)} * ({round(gp[1],3)}^x)"])
#ax.figure.savefig(f'archive/{RUN_DATE}/growth-rate-trend.png')

In [ ]:
df=select_outbreak(project_ols_growth_rate_min(vic_partial, 110, gp[1]))
ax=plot_derivatives(df, len(vic_partial), dataset="Melbourne 2020")
#ax.figure.savefig(f'archive/{RUN_DATE}/derivatives-melbourne-partial.png')

In [ ]:
ax=plot_derivatives(vic_outbreak, None, dataset="Melbourne 2020")
ax.figure.savefig(f'archive/{RUN_DATE}/derivatives-melbourne-full.png')

# Hedgehog Animation

In [ ]:
%%capture phaseplot
animate_phaseplot(
    df=bondi, 
    outbreak="Sydney 2021", 
    fn=f'archive/{RUN_DATE}/animated-hedgehog-sydney2021.gif', 
    offset=15
)
if ENABLE_MELBOURNE_ANIMATION:
    animate_phaseplot(
        df=vic_outbreak, 
        offset=20, 
        outbreak="Melbourne 2020", 
        fn=f'archive/{RUN_DATE}/animated-hedgehog-melbourne2020.gif'
    )

In [ ]:
display(HTML(f"<img src='animated-hedgehog-sydney2021.gif'>"))

In [ ]:
display(HTML(f"<img src='../latest/animated-hedgehog-melbourne2020.gif'>"))

# Derivatives Animation

In [ ]:
%%capture derivatives
animate_derivatives(
    bondi, 
    "Sydney 2021", 
    f'archive/{RUN_DATE}/animated-derivatives-sydney.gif', 
    derive_growth_params(bondi[(bondi.index>15)])[1]
)

if ENABLE_MELBOURNE_ANIMATION:
    animate_derivatives(
        vic_outbreak, 
        "Melbourne 2020", 
        f'archive/{RUN_DATE}/animated-derivatives-melbourne.gif', 
        derive_growth_params(vic_outbreak[(vic_outbreak.index>30)])[1]
    )
                    
                    

In [ ]:
display(HTML(f"<img src='animated-derivatives-sydney.gif'>"))

In [ ]:
display(HTML(f"<img src='../latest/animated-derivatives-melbourne.gif'>"))

# Log Log Plots

The minutephysics YouTube channel popularised the technique of using [log-log](https://www.youtube.com/watch?v=54XLXg4fYsc&vl=en) plots to visualise when exponential growth of daily case totals slows and then reverses.

This section creates static and animated log-log plots. In addition to plotting the log-log curve, we fit trend lines to the last 14 days to better help visualise changes in growth rates. This is most evident in the animated plots. 



## static log-log plot

In [ ]:
ax=plot_log_log_day(bondi.index.max(), vic_outbreak, bondi)
ax.figure.savefig(f"archive/{RUN_DATE}/log-log.png")

## log-log gradient vs time
Also, we plot the calculated gradients against time to see how they change with time.

Note: we calculate and plot the gradient of the natural log-log curves not the log base 10 curves.

In [ ]:
ax=bondi["ltlc-gradient"].plot(figsize=(10,10))
ax.plot(vic_outbreak["ltlc-gradient"])
ax.hlines(0,xmin=0,xmax=80, color="black", linestyle="dotted")
ax.set_ylim(top=4, bottom=-2.5)
ax.set_xlim(left=0, right=80)
ax.set_ylabel("ln-ln gradient")
ax.set_xlabel("day of outbreak")
ax.set_title("gradient of ln daily cases vs ln cumulative cases plot vs day of outbreak")
ax.grid()
ax.figure.savefig(f"archive/{RUN_DATE}/ltlc-gradients.png")

# df=vic_outbreak[vic_outbreak.index < 60]
# df=bondi
# endog=df['ltlc-gradient']
# exog=sm.add_constant(df.index)
# window=31
# model = RollingOLS(endog=endog, exog=exog, window=window, min_nobs=window)
# params=model.fit().params.tail(1)
# ax.plot((vic_outbreak.index*params["x1"].values[0]+params["const"].values[0]))


## animated log-log plot

In [ ]:
%%capture log_log
animate_log_log_plot(vic_outbreak, bondi, f"archive/{RUN_DATE}/animated-log-log.gif")

In [ ]:
display(HTML(f"<img src='animated-log-log.gif'>"))